# Gene Regulatory Networks Reveal Sex differences in Lung Adenocarcinoma
Authors: Enakshi Saha<sup>1</sup>, Marouen Ben Guebila<sup>1</sup>, Camila M. Lopes-Ramos <sup>1,2,3</sup>

<sup>1</sup>Department of Biostatistics, Harvard T.H. Chan School of Public Health, Boston, MA.

<sup>2</sup>Channing Division of Network Medicine, Brigham and Women's Hospital, Boston, MA.

<sup>3</sup>Department of Medicine, Harvard Medical School, Boston, MA.

## 1. Introduction 

Lung adenocarcinoma (LUAD) has been observed to have significant sex differences in incidence risk, prognosis and response to therapy. While females have much higher chance of developing the disease compared to males, males have poorer survival outcome[1]. The molecular mechanisms responsible for these disparities have not been investigated extensively. 
We use sample-specific gene regulatory network (GRN) methods PANDA[2] and LIONESS[3] to construct sample-specific GRNs using RNA sequencing data from lung adenocarcinoma primary tumor samples from The Cancer Genome Atlas (TCGA)[4] to identify genes and biological processes that are differentially targeted by transcription factors (TF) between males and femlaes. Uniformly processed RNA-Seq data along with information on clinical covariates for LUAD tumors from TCGA were downloaded from the Recount3 database on May 26, 2022.

PANDA and LIONESS algorithms were used to construct individual sample-specific gene regulatory networks using Python package netZooPy version 0.9.10[7]. In addition to the gene expression data, two other data sources were utilized to construct the regulatory networks: Transcription factor/target gene regulatory prior (derived by mapping Transcription factor motifs from the Catalog of Inferred Sequence Binding Preferences (CIS-BP)[8] to the promoter of target genes) and protein-protein interaction (using the interaction scores from StringDb v11.5[9] between all Transcription factor in the regulatory prior).

The networks are publicly available on the GRAND database[5]: https://grand.networkmedicine.org/downloads/ and can be found under the search term “sexDiffLUAD”. In this notebook we compare the indegree (i.e. the sum of all incoming edges from all TFs to a particular gene) of genes computed for every sample-specific network from males and females, while adjusting for clinical covariates: age at diagnosis, race, smoking status and tumor stage. We perform a Gene Set Enrichment Analysis (GSEA) using the list of biological processes obtained from the Kyoto Encyclopedia of Genes and Genomes (KEGG) database[6].

### Loading libraries and data

The following libraries are used in this analysis:

In [ ]:
library(data.table)   # To load indegree of genes for every sample-specific network
library(Biobase)      # Needed for limma
library(limma)        # To perform differential targeting analysis
library(fgsea)        # To perform GSEA
library(ggplot2)      # To plot GSEA results
library(gplots)       # To plot GSEA results
library(RColorBrewer) # To plot GSEA results

Let us first set the project path to load network indegrees. These indegree matrices are precomputed and also available on the GRAND database https://grand.networkmedicine.org/downloads/.

In [ ]:
ppath = '/opt/data/netZooR/luadsexdiffen/'

Here we will load two matrices, containing the indegree of genes for males and females respectively. For each matrix, the rows correspond to genes (annotated by Ensemble IDs) and the columns correspond to samples.

In [ ]:
# Load indegree matrices for males and females
indegree_male = data.frame(fread(paste0(ppath,"TCGA_lung_lioness_indegree_male.txt")))
indegree_female = data.frame(fread(paste0(ppath,"TCGA_lung_lioness_indegree_female.txt")))

# Get ensemble ID of genes
genes = indegree_male$V1

# Combine male and female indegrees into a single matrix.
indegree = cbind(indegree_male[,-1], indegree_female[,-1])
rownames(indegree) = genes


In [ ]:
sample_IDs = colnames(indegree)
# Format sample IDs to match with phenotypic data
sample_IDs[which(substring(sample_IDs, 1,1) == "X")] = substring(sample_IDs[which(substring(sample_IDs, 1,1) == "X")], 2, length(sample_IDs[which(substring(sample_IDs, 1,1) == "X")]))
sample_IDs = unlist(lapply(strsplit(sample_IDs, split=".", fixed = T),
                           function(x){paste(x, collapse ="-")}))

Next we load a matrix containing chromosome locations and HGNC gene names (required for GSEA) for all genes.

In [ ]:
# Get chromosome and gene names
gene_info = data.frame(fread(paste0(ppath,"gene_info.txt")))
gene_info$gene_id = gsub("\\..*","",gene_info$gene_id)

# Extract HGNC gene names for each gene in the indegree matrix
gene_name = gene_info$gene_name[match(genes, gene_info$gene_id)]

# Extract chromosome location for each gene in the indegree matrix
chr_loc = gene_info$seqnames[match(genes, gene_info$gene_id)]

We exclude the Y genes from our analysis since females do not have Y chromosome.

In [ ]:
# Remove Y genes
indegree = indegree[-which(chr_loc == "chrY"),]
genes = genes[-which(chr_loc == "chrY")]
rownames(indegree) = genes

Next we load a matrix containing phenotypic information for all samples.

In [ ]:
# Get phenotypes
phenotypes = data.frame(read.csv(paste0(ppath,"TCGA_phenotypes.txt", sep="")))
phenotypes = phenotypes[match(sample_IDs, rownames(phenotypes)),]
phenotypes = phenotypes[,c("tcga.gdc_cases.demographic.gender", "tcga.gdc_cases.demographic.race", 
                                "tcga.xml_age_at_initial_pathologic_diagnosis", "tcga.gdc_cases.samples.sample_type", 
                                "tcga.gdc_cases.diagnoses.tumor_stage", "tcga.xml_tobacco_smoking_history")]
colnames(phenotypes) = c("sex", "race", "age", "sample_type", "tumor_stage", "smoking_status")
head(phenotypes)

Next we specify factor levels for each categorical covariate and impute missing values for numerical covariates.

In [ ]:
# Define the covariates: gender, age, race, smoking, tumor_stage

sex = phenotypes$sex
sex[which(sex == "male")] = "MALE" 
sex[which(sex == "female")] = "FEMALE" 
sex = factor(sex, levels = c("MALE", "FEMALE"))

race = phenotypes$race
race[which(race != "black or african american" & race != "white")] = "others"
race = factor(race)

age <- as.numeric(as.character(phenotypes$age))
age[which(is.na(age))] = mean(age,na.rm=TRUE)

smoking_status = phenotypes$smoking_status
smoking_status[which(is.na(smoking_status))] = "Unknown"
smoking_status[which(smoking_status == 1)] = "No"
smoking_status[which(smoking_status %in% 2:5)] = "Yes"
smoking_status = factor(smoking_status, levels = c("No", "Yes", "Unknown"))

tumor_stage = phenotypes$tumor_stage
tumor_stage[which(tumor_stage == "stage i" | tumor_stage == "stage ia" | tumor_stage == "stage ib")] = "stageI"
tumor_stage[which(tumor_stage == "stage ii" | tumor_stage == "stage iia" | tumor_stage == "stage iib")] = "stageII"
tumor_stage[which(tumor_stage == "Stage iii" | tumor_stage == "stage iiia" | tumor_stage == "stage iiib")] = "stageIII"
tumor_stage[which(tumor_stage == "stage iv")] = "stageIV"
tumor_stage = factor(tumor_stage, levels = c("stageI", "stageII", "stageIII", "stageIV", "not reported"))


## 2. Differential Targeting Analysis

Next we construct the design matrix for differential targeting analysis and fit linear model with R package "limma" for quantifying the effect of sex for smokers and nonsmokers separately, while adjusting for age, race and tumor stage.

In [ ]:
# Define the design matrix
design = model.matrix(~ age + sex + race + smoking_status + tumor_stage + sex*smoking_status)

# Fit linear model
fit <- lmFit(indegree, design)
fit <- eBayes(fit)

We order genes by descending order of difference between females and males, so that genes highly targeted in females compared to males are at the top of the table and genes highly targeted in males compared to females are at the bottom of the table.

In [ ]:
# Table for sex difference: nonsmoker
tb = topTable(fit,coef="sexFEMALE",number=Inf)
tb$chr = gene_info$seqnames[match(rownames(tb), gene_info$gene_id)]
tb$gene_name = gene_info$gene_name[match(rownames(tb), gene_info$gene_id)]
head(tb)
tb_nonsmoker = tb

# Table for sex difference: smoker
colnames(design) = make.names(colnames(design))
contrast.matrix <- makeContrasts(sexFEMALE_smoker = sexFEMALE + sexFEMALE.smoking_statusYes, levels=design)
colnames(fit$coefficients) = rownames(contrast.matrix)
fit2 <- contrasts.fit(fit, contrast.matrix)
fit2 <- eBayes(fit2)
tb = topTable(fit2,coef="sexFEMALE_smoker",number=Inf)
tb$chr = gene_info$seqnames[match(rownames(tb), gene_info$gene_id)]
tb$gene_name = gene_info$gene_name[match(rownames(tb), gene_info$gene_id)]
head(tb)
tb_smoker = tb

## 3. Gene set enrichment analysis
We rank genes by the t-statistic of the sex coefficient and perform GSEA using these ranked list of genes.

In [ ]:
# Rank genes in limma table
indegree_rank_nonsmoker <- setNames(object=tb_nonsmoker[,"t"], tb_nonsmoker$gene_name)
head(indegree_rank_nonsmoker)

indegree_rank_smoker <- setNames(object=tb_smoker[,"t"], tb_smoker$gene_name)
head(indegree_rank_smoker)

In [ ]:
# Load KEGG pathways
pathways <- gmtPathways(paste0(ppath,"GSEA_pathways/c2.cp.kegg.v2022.1.Hs.symbols.gmt"))

In [ ]:
# Perform GSEA
fgseaRes_nonsmoker <- fgsea(pathways, indegree_rank_nonsmoker, minSize=15, maxSize=500)
fgseaRes_smoker <- fgsea(pathways, indegree_rank_smoker, minSize=15, maxSize=500)

We demonstrate a heatmap of all biological pathways that are significantly differentially targeted between males and females in either nonsmokers or smokers.

In [ ]:
# Create a table with NES scores by smoking status for drawing Heatmap
heatmap_tab = cbind(fgseaRes_nonsmoker$NES, fgseaRes_smoker$NES[match(fgseaRes_nonsmoker$pathway, fgseaRes_smoker$pathway)])
rownames(heatmap_tab) = fgseaRes_nonsmoker$pathway
head(heatmap_tab)

# Plot only Pathways significant at level 0.05 in either nonsmokers or smokers
sig_pathways = rownames(heatmap_tab)[which((fgseaRes_nonsmoker$padj<0.05) | (fgseaRes_smoker$padj<0.05))]
tab_subset = heatmap_tab[which(rownames(heatmap_tab) %in% sig_pathways),]

colnames(tab_subset) = c("nonsmoker", "smoker")
rownames(tab_subset) = stringr::str_to_title(lapply(strsplit(rownames(tab_subset), split = "_"), function(x){paste(x[-1], collapse = " ")}))


In [ ]:
mycol2 <- colorRampPalette(rev(brewer.pal(11,"RdBu")))(50)
heatmap_tcga = heatmap.2(as.matrix(tab_subset),density.info="none",trace="none",col=mycol2,symbreaks=T,symkey=T, 
                                cexRow=1.5, cexCol=1.5, srtCol = 0, keysize=0.5, mar=c(20,50), key.title=NULL, key.xlab="NES", Colv = F)

## 4. Conclusion 

Based on our analysis we observe that in lung adenocarcinoma primary tumor, pathways related to cell adhesion and cell proliferation including pathways in cancer, WNT signaling pathway and MTOR signaling pathway were highly targeted in males, compared to females irrespective of smoking status.

Most pathways associated to immune response including allograft rejection, intestinal immune response for IGA production, antigen processing and presentation, Natural Killer Cell Mediated Cytotoxicity, all showed higher targeting in females compared to males in both nonsmokers and smokers. However, other immune associated pathways such as Jak-Stat signaling pathway, Chemokine Signaling Pathway and Cytokine Cytokine Receptor Interaction were highly targeted in males among nonsmokers and in females among smokers.

This differential regulation of key biological pathways might be a contributing factor towards the difference in prognosis and response to therapy between males and females in LUAD.

## References 

1- Lopes-Ramos, Camila M., John Quackenbush, and Dawn L. DeMeo. "Genome-wide sex and gender differences in cancer." Frontiers in Oncology 10 (2020): 2486

2- Glass, Kimberly, et al. "Passing messages between biological networks to refine predicted interactions." PloS one 8.5 (2013): e64832.

3- Kuijjer, Marieke Lydia, et al. "Estimating sample-specific regulatory networks." Iscience 14 (2019): 226-240.

4- Tomczak, Katarzyna, Patrycja Czerwińska, and Maciej Wiznerowicz. "The Cancer Genome Atlas (TCGA): an immeasurable source of knowledge." Contemporary oncology 19.1A (2015): A68.

5- Ben Guebila, Marouen, et al. "GRAND: a database of gene regulatory network models across human conditions." Nucleic Acids Research 50.D1 (2022): D610-D621.

6- Kanehisa, Minoru. "The KEGG database." ‘In silico’simulation of biological processes: Novartis Foundation Symposium 247. Vol. 247. Chichester, UK: John Wiley & Sons, Ltd, 2002.

7- Ben Guebila, Marouen, et al. "The Network Zoo: a multilingual package for the inference and analysis of gene regulatory networks." Genome Biology 24.1 (2023): 45.

8- Lambert, Samuel A., et al. "The human transcription factors." Cell 172.4 (2018): 650-665.

9- Szklarczyk, Damian, et al. "The STRING database in 2021: customizable protein–protein networks, and functional characterization of user-uploaded gene/measurement sets." Nucleic acids research 49.D1 (2021): D605-D612.